In [1]:
import numpy as np
import os
from thorlabs_tsi_sdk.tl_camera import TLCameraSDK, OPERATION_MODE
import matplotlib.pyplot as plt
import tifffile
import time

output_dir = './output_cam_test/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

NUM_FRAMES = 30  # adjust to the desired number of frames
os.add_dll_directory(os.path.abspath('../imswitch/imcontrol/model/interfaces/thorlabs_tsi_sdk/dll/')) # add the location of the SDK DLLs to the PATH
filename = 'output.tif'
# check if filename exists, if so, delete it
if os.path.exists(output_dir + os.sep + filename):
    os.remove(output_dir + os.sep + filename)
    
with TLCameraSDK() as sdk:
    available_cameras = sdk.discover_available_cameras()
    if len(available_cameras) < 1:
        print("no cameras detected")
    sdk.open_camera(available_cameras[0])
    with sdk.open_camera(available_cameras[0]) as camera:
        camera.exposure_time_us = 10000  # set exposure to 11 ms
        camera.operation_mode = 1
        camera.frames_per_trigger_zero_for_unlimited = 1  # start camera in continuous mode
        camera.image_poll_timeout_ms = 2000  # 1 second polling timeout

        camera.arm(2)
        t0 = time.time()
        camera.issue_software_trigger()
        frames_acquired = 0
        while frames_acquired < NUM_FRAMES:
            frame = camera.get_pending_frame_or_null()
            frames_acquired += 1
            if frame is not None:
                print("frame #{} received!".format(frame.frame_count))
                img = frame.image_buffer
                with tifffile.TiffWriter(output_dir + os.sep + filename, append=True) as tif:
                    tif.write(data = img)
                    

            else:
                raise TimeoutError("Timeout was reached while polling for a frame, program will now exit")

        t = time.time() - t0
        print('Acqusition done, time elapsed: {:.2f} s and {:.2f} ms per image'.format(t, t/NUM_FRAMES*1000))
                
            
        camera.disarm()

#  Because we are using the 'with' statement context-manager, disposal has been taken care of.

print("program completed")

frame #1 received!
frame #2 received!


KeyboardInterrupt: 

In [ ]:
import tifffile
path = 'C:/Users/alm/Documents/ImSwitchConfig/recordings/2024-05-26/13h55m28s_rec_thorlabscam.tiff'
tif = tifffile.imread(path)
print(tif.shape)